# Evaluating Files declared in code


This tutorial highlights how to combine Humanloop decorated files.


### 1. Setting up imports


In [1]:
import os

from dotenv import load_dotenv
from humanloop import Humanloop
import chromadb
from openai import OpenAI
import pandas as pd

from levenshtein import compare_log_and_target
from exact_match import exact_match


load_dotenv()

True

### 2. Instantiating the Humanloop client


In [ ]:
humanloop = Humanloop(api_key=os.getenv("HL_API_KEY"))

### 3. Instantiating the vector database


In [3]:
chroma = chromadb.chromadb.Client()
collection = chroma.get_or_create_collection(name="MedQA")

# init collection into which we will add documents
knowledge_base = pd.read_parquet("../../../assets/sources/textbooks.parquet")
knowledge_base = knowledge_base.sample(5, random_state=42)
collection.add(
    documents=knowledge_base["contents"].to_list(),
    ids=knowledge_base["id"].to_list(),
)

### 4. Loading the evaluation dataset


In [4]:
datapoints_df = pd.read_json("../../../assets/datapoints.jsonl", lines=True)
datapoints = [row.to_dict() for _, row in datapoints_df.iterrows()][:20]

### 5. Declare Humanloop Files via code


In [5]:
TEMPLATE = """Answer the following question factually.

Question: {question}

Options:
- {option_A}
- {option_B}
- {option_C}
- {option_D}
- {option_E}

---

Here is some retrieved information that might be helpful.
Retrieved data:
{retrieved_data}

---

Give you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.
```
<chosen option verbatim>
---
<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>
---
<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>
```
"""

In [6]:
import json


def retrieval_tool(question: str) -> str:
    """Retrieve most relevant document from the vector db (Chroma) for the question."""
    response = collection.query(query_texts=[question], n_results=1)
    retrieved_doc = response["documents"][0][0]
    return retrieved_doc


def ask_model(
    question: str,
    option_A: str,
    option_B: str,
    option_C: str,
    option_D: str,
    option_E: str,
) -> str:
    """Ask a question and get an answer using a simple RAG pipeline"""
    openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Retrieve context
    retrieved_data = retrieval_tool(question)
    inputs = {
        "question": question,
        "option_A": option_A,
        "option_B": option_B,
        "option_C": option_C,
        "option_D": option_D,
        "option_E": option_E,
        "retrieved_data": retrieved_data,
    }

    # Populate the Prompt template
    messages = [
        {
            "role": "user",
            "content": TEMPLATE.format(**inputs),
        }
    ]

    # Call OpenAI to get response
    chat_completion = openai.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=messages,
    )
    return chat_completion.choices[0].message.content


def entrypoint(
    question: str,
    option_A: str,
    option_B: str,
    option_C: str,
    option_D: str,
    option_E: str,
):
    output = ask_model(
        question=question,
        option_A=option_A,
        option_B=option_B,
        option_C=option_C,
        option_D=option_D,
        option_E=option_E,
    )
    humanloop.flows.log(
        path="Evaluations SDK Demo/MedQA Answer Flow 8",
        flow={
            "attributes": {
                "prompt": {
                    "model": "gpt-4o",
                    "environment": "evaluation",
                }
            }
        },
        inputs={
            "question": question,
            "option_A": option_A,
            "option_B": option_B,
            "option_C": option_C,
            "option_D": option_D,
            "option_E": option_E,
        },
        output=json.dumps({"answer": output}),
    )
    return output

### 6. Evaluate the Flow


In [7]:
humanloop.evaluations.run(
    file={
        "path": "Evaluations SDK Demo/MedQA Answer Flow 8",
        "callable": entrypoint,
        "version": {
            "attributes": {
                "prompt": {
                    "model": "gpt-4o",
                    "environment": "evaluation",
                }
            }
        },
        "type": "flow",
    },
    name="MedQA Evaluation Decorators Debug",
    dataset={
        "datapoints": datapoints,
        "path": "Evaluations SDK Demo/Dataset",
    },
    evaluators=[
        {
            "path": "Evaluations SDK Demo/Levenshtein",
            "args_type": "target_required",
            "return_type": "number",
            "callable": compare_log_and_target,
        },
        {
            "path": "Evaluations SDK Demo/Exact Match",
            "args_type": "target_required",
            "return_type": "boolean",
            "callable": exact_match,
        },
    ],
    workers=8,
)

Evaluating your flow function corresponding to `Evaluations SDK Demo/MedQA Answer Flow 8` on Humanloop 



Navigate to your Evaluation:
http://localhost:3000/project/fl_iiAjv1p4Kt7BUODEpsIro/evaluations/evr_Fe6hBqvelFS6mJCzmIqlw/stats

Flow Version ID: flv_jehMg9qF0zyxvLgE2bweR
Run ID: rn_rEKHLYePrD4iTYO3ZjT5i

Running 'MedQA Answer Flow 8' over the Dataset 'Dataset' using 8 workers 
[##--------------------------------------] 1/20 (5.00%) | ETA: 0s

SHOULD BE NONE None


[####------------------------------------] 2/20 (10.00%) | ETA: 3s

SHOULD BE NONE None


[######----------------------------------] 3/20 (15.00%) | ETA: 13s

...

SHOULD BE NONE None


[########################################] 20/20 (100.00%) | DONE


SHOULD BE NONE None



⏳ Evaluation Progress
Total Logs: 20
Total Judgments: 39



⏳ Evaluation Progress
Total Logs: 20
Total Judgments: 40



📊 Evaluation Results for Evaluations SDK Demo/MedQA Answer Flow 8 
+----------------------------------+---------------------+
|                                  |        Latest       |
+----------------------------------+---------------------+
|                           Run ID |        rEKHL        |
+----------------------------------+---------------------+
|                       Version ID |         None        |
+----------------------------------+---------------------+
|                            Added | 2024-11-11 14:27:49 |
+----------------------------------+---------------------+
|                       Evaluators |                     |
+----------------------------------+---------------------+
| Evaluations SDK Demo/Levenshtein |        22.85        |
| Evaluations SDK Demo/Exact Match |         0.0         |
+----------------------------------+---------

[]